<h1>Exploratory Data Analysis</h1>

In [30]:
import pandas as pd
import datetime as dt
import warnings
warnings.filterwarnings('ignore', category=UserWarning, module='openpyxl')

In [8]:
data_xlsx_path = '../data/TEIS_IFSP_Teaming_Jan_2022-_Jan_2023.xlsx'
ifsp_teaming_df = pd.read_excel(data_xlsx_path, sheet_name='Compiled_IFSP Teaming')
all_children_ifsp_df = pd.read_excel(data_xlsx_path, sheet_name='All Children with IFSPs')

<h3>1. How many children are receiving co-visits? What is the rate of co-visits? That is, out of children who have an IFSP, how many are receiving co-visits?</h3>

In [77]:
# Getting the distinct number of children with IFSPs
n_unique_children = all_children_ifsp_df['Child ID'].nunique()

# Getting all of the covisit records
covisit_df = ifsp_teaming_df[ifsp_teaming_df['IFSP Teaming ID name'] == 'Co-Visit']
n_covisits = len(covisit_df)

# Declaring a set to hold the unique children with covisit records
unique_children_covisit = set()

# Looping through the covisit records and adding the unique children to the set
for index, row in covisit_df.iterrows():
    unique_children_covisit.add(row['CHILD_ID'])

n_unique_children_receiving_covisit = len(unique_children_covisit)

rate_of_covisits = round(n_unique_children_receiving_covisit / n_unique_children, 4)

print(f'Number of unique children receiving co-visits: {len(unique_children_covisit)}')
print(f'The rate of unique children receiving co-visits: {rate_of_covisits} or {rate_of_covisits * 100}%')

Number of unique children receiving co-visits: 1645
The rate of unique children receiving co-visits: 0.0838 or 8.38%


<h3>2. Children can have up to 6 co-visits per year. How many are recieving all 6 visits? Are there any who are exceeding 6 visits? In what setting do co-visits typically occur?</h3>

In [76]:
n_children_receiving_6_visits_per_year = 0
n_children_receiving_more_than_6_visits_per_year = 0
children_covisit_insights = {}
service_settings = {
    'community_based': 0,
    'error': 0,
    'home': 0,
    'other': 0,
    'telehealth': 0
}

for index, row in covisit_df.iterrows():
    # excluding any records where the date is in the month january 2022 in order to only use 1 year of data
    year_month = row['SERVICE_DATE'].strftime('%Y-%m')
    if year_month == '2022-01': continue

    # Building the dictionary entry
    child_id = row['CHILD_ID']
    if child_id not in children_covisit_insights:
        children_covisit_insights[child_id] = {
            'n_visits': 1,
            '6_or_more_visits': False,
        }
    else:
        children_covisit_insights[child_id]['n_visits'] += 1

        if children_covisit_insights[child_id]['n_visits'] == 6:
            n_children_receiving_6_visits_per_year += 1
        elif children_covisit_insights[child_id]['n_visits'] > 6:
            # If a child is receiving more than 6 visits per year, we only want to count them once
            if children_covisit_insights[child_id]['6_or_more_visits'] == False:
                n_children_receiving_more_than_6_visits_per_year += 1
                children_covisit_insights[child_id]['6_or_more_visits'] = True

    # Incrementing the appropriate service setting
    service_setting = str(row['SERVICE_SETTING']).lower()
    if service_setting == 'community-based':
        service_settings['community_based'] += 1
    elif service_setting == 'home':
        service_settings['home'] += 1
    elif service_setting == 'other':
        service_settings['other'] += 1
    elif service_setting == 'telehealth':
        service_settings['telehealth'] += 1
    else:
        service_settings['error'] += 1

# How many children are receiving no visits
n_children_receiving_no_visits = n_unique_children - len(children_covisit_insights)
print(f'Number of unique children: {n_unique_children}')

# 

# Insights
print(f'Number of children receiving 6 visits per year: {n_children_receiving_6_visits_per_year}')
print(f'Number of children receiving more than 6 visits per year: {n_children_receiving_more_than_6_visits_per_year}')
print(f'The most common service setting is: {max(service_settings, key=service_settings.get)}')
print(f'Number of children receiving no visits: {n_children_receiving_no_visits}')
# print(children_covisit_insights)
# print(service_settings)

Number of unique children: 19623
Number of children receiving 6 visits per year: 121
Number of children receiving more than 6 visits per year: 87
The most common service setting is: home
Number of children receiving no visits: 18062


<h3>3. Who is performing co-visits? Are they generally performed by the same provider or discipline or by multiple providers or disciplines? Is it more likely for a co-visit to occur if vendor services are within the same agency?</h3>

In [74]:
# Get all of the unique agency names from the covisit dataframe and store in a list
unique_agency_names = covisit_df['Agency Name'].unique().tolist()

# Use the above list to create keys in a dictionary
agency_frequency_dict = {}
for agency_name in unique_agency_names:
    agency_frequency_dict[agency_name] = 0

# Loop through the covisit dataframe and increment the appropriate agency name key
for index, row in covisit_df.iterrows():
    agency_name = row['Agency Name']
    agency_frequency_dict[agency_name] += 1

# Print the most frequent agency names to least frequent
print(sorted(agency_frequency_dict.items(), key=lambda x: x[1], reverse=True))

[('LeBonheur EI & Dev (LEAD) (vendor)', 902), ('LeBonheur EI & Dev (LEAD) (EIRA)', 340), ('Shelby Residential & Voc Svcs(SRVS) (EIRA)', 232), ('Therapy & Learning Center, Inc. (EIRA)', 216), ("Kid's Putnam, Inc.", 212), ('Physical Therapy Services, PA (EIRA)', 181), ('WAVES, Inc.', 169), ("Siskin Children's Institute (EIRA)", 149), ('Emory Valley Center', 138), ('Expanding Horizons - ETSU', 130), ("Erlanger Health Care System/TC Thompson's Children", 127), ('Michael Dunn Center/ The Henry Center', 112), ('Cleveland TN Hosp dba Tennova Healthcare-Cleveland', 111), ('Signal Centers, Inc.', 88), ('Rainbow Early Intervention (HATS)', 81), ('Middle Tennessee State University', 76), ('Sunshine Early Intervention - Knox Cty ARC', 66), ('Therapeutic Rehab Services - Cathy Otis', 57), ('LTVEC', 56), ('BEST (Blind-Visually Impaired Early Svcs Tn)', 44), ('Progressive Directions Inc.', 42), ('Therapy & Learning Center Inc (vendor WTCCD)', 41), ('Morristown Hamblen Child Care Center', 39), ('Commu

<h3>4. Does participation vary by service type? by service setting? by point of entry?</h3>